In [17]:
from bm_preproc import BoyerMoore
from bm_preproc import z_array

In [54]:
def get_index_hits (p, p_bm, t):
    char_comp= 0
    align_try= 0
    index_hits= []
    t_idx= 0
    while t_idx <= len(t)-len(p)+1:
        align_try += 1
        #print('t_idx %i' %t_idx)
        for p_idx in range(len(p)-1,-1,-1):
            char_comp += 1
            if t[t_idx+p_idx] != p[p_idx]:
                bc_skip= p_bm.bad_character_rule(i=p_idx, c=t[t_idx+p_idx])
                gs_skip= p_bm.good_suffix_rule(p_idx)
                shift= max(bc_skip, gs_skip)
                #print('  Mismatch %i, %i' % (t_idx, p_idx))
                break
        if t[t_idx:t_idx+len(p)]== p:
            print('Match %i' % t_idx)
            index_hits.append(t_idx)
            shift= p_bm.match_skip()
        t_idx += shift
    return [index_hits, char_comp, align_try]

pat= 'ACT'
query= 'ACTG'
tex= 'TCACTGACT' #'TCACTG'
bmObj= BoyerMoore(p= pat)
[idx_hits, char_comp, align_try]= get_index_hits(pat, bmObj, tex)
print('Index hits:', idx_hits)
print('Character comparisons: %i' %char_comp)
print('Alignments tried: %i' %align_try)

Match 2
Match 6
Index hits: [2, 6]
Character comparisons 8
Alignments tried 4


In [42]:
def verify(index_hits, t, query, offset=0):
    #offset: how far patern it to the right of query start
    matchPos= []
    for ih in index_hits:
        #print('index-hit: %i' %ih)
        if t[(ih-offset):(ih+len(query)-offset)] == query:
            #print('Check match %s vs query %s' % (t[(ih-offset):(ih+len(query)-offset)], query))
            matchPos.append(ih-offset)
    return matchPos

print( verify(idx_hits, tex, query) )

index-hit: 2
Check match ACTG vs query ACTG
index-hit: 6
[2]


In [56]:
def naive_exact(p,t):
    char_comp= 0
    align_try= 0
    matches= []
    for t_idx in range(len(t)-len(p)+1):
        align_try += 1
        #print('t index: %i' %t_idx)
        match= 1
        for p_idx in range(len(p)):
            char_comp += 1
            #print(' p index: %i' %p_idx)
            if t[t_idx+p_idx] != p[p_idx]:
                match= 0
                break
        if match == 1:
            #print('Match: %i' %t_idx)
            matches.append(t_idx)
    return([matches, char_comp, align_try])

[naiveExact_matches, char_comp, align_try]= naive_exact(query, tex)
print('Matches:', naiveExact_matches )
print('Character comparisons: %i' %char_comp)
print('Alignments tried: %i' %align_try)

Matches: [2]
Character comparisons: 9
Alignments tried: 6


In [67]:
genome= ''
f= open('chr1.GRCh38.excerpt.fasta')
while True:
    l= f.readline()
    l= l.rstrip()
    if len(l) == 0:
        break
    if l[0] != '>':
        genome += l

print( genome[:100] )
print('length %i' %len(genome))

TTGAATGCTGAAATCAGCAGGTAATATATGATAATAGAGAAAGCTATCCCGAAGGTGCATAGGTCAACAATACTTGAGCCTAACTCAGTAGATCCTAAAA
length 800000


In [68]:
# 1,
query= 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
[naiveExact_matches, char_comp, align_try]= naive_exact(query, genome)
print('Alignments %i' %align_try)

#2,
print('Character comparisons %i' %char_comp)

Alignments 799954
Character comparisons 984143


In [70]:
#3,
pat= 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
tex= genome
bmObj= BoyerMoore(pat)
[idx_hits, char_comp, align_try]= get_index_hits(pat, bmObj, tex)
print('Index hits:', idx_hits)
print('Character comparisons: %i' %char_comp)
print('Alignments tried: %i' %align_try)

Match 56922
Index hits: [56922]
Character comparisons: 165191
Alignments tried: 127974


In [85]:
# Pigeonhole
 #pattern has length 24
 #allow two MM

from kmer_index import Index

genome= 'GGCGCGGTGGCTCACGCCTGTAAT'
pat= 'AAAGGCGCGGTGGCTCACGCCTGTAATAAA'
iObj= Index(genome, 8)

frag_cuts= range(0,25,8)
match_pos= {}
for i in range(1,len(frag_cuts)):
    print(i)
    frag= pat[frag_cuts[i-1]:frag_cuts[i]]
    print('frag: %s' %frag)
    
    fragMatches= iObj.query(frag) #matches of fragment
    if( len(fragMatches)== 0):
        continue

    for fM_no in fragMatches:
        mm_count= 0
        # look left
        left_end= fM_no - i*8
        if left_end < 0 or left_end+len(pat)>len(genome):
            continue
        for j in range(i*8):
            if genome[left_end+j] != pat[j]:
                mm_count += 1
            if mm_count > 2:
                break
        #look right
        #right_start= fM_no+8
        for j in range(len(pat)-i*8,len(pat)):
            if genome[left_end+j] !=  pat[j]:
                mm_count += 1
            if mm_count > 2:
                break
        if mm_count <= 2:
            match_pos.append(fragMatches[fM_no] - i*8)

print("Full match positions:", match_pos)

1
frag: AAAGGCGC
2
frag: GGTGGCTC
3
frag: ACGCCTGT
Full match positions: {}
